In [2]:
import os
import pandas as pd
import jieba  
import xlrd
import xlwt 
from collections import Counter
import re
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2#卡方检验——特征筛选
import numpy as np
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, ShuffleSplit
import lightgbm as lgb
import xgboost as xgb
import catboost as cbt
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
os.chdir(r"C:\Users\Administrator\OneDrive\课件\自然语言处理\分类")

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '

In [3]:
data=pd.read_csv('10000_hotel.csv', encoding='ansi')

In [4]:
data

,comment,sentiment
0,"首先感觉不像大酒店,应该属于公寓式酒店,房间很小,明显是住宅公寓改装的.其次门童的服务不好,...",neg
1,房间和走廊里味道很重，楼外下面马路上的声音很吵，早上隔壁的电视机的声音听的很清楚，可以当闹钟...,neg
2,虽然预定比较方便，但是宾馆的对顾客的关怀不足，房间冷得让人睡不着觉服务人员还说暖和的！！！宾...,neg
3,酒店位置不是很理想，去金元购物中心还是蛮近的，大约10分钟车程，我入住的是448的房间，正赶...,neg
4,真不知道之前的点评是怎么得出来的，还以为会是一个不错的酒店，谁知完全不是那么回事。首先酒店没...,neg
5,号称平遥最高级的四星酒店，实际只是二星半的硬件，二十年前国营招待所的服务，差极了。可能因为住...,neg
6,根本就是一个招待所！亏的还能上携程网推荐！真是丢死人的脸了。前台（姑且升格称其为前台）服务态...,neg
7,酒店的管理不专业，服务非常差，房间的空调不足，冬天的被子象夏天被，晚上住宿要用自己的羽绒衣防寒。,neg
8,订了商务大床间，可惜房间小的可怜！床和写字台之间太紧，基本上不能转身。更无法忍受的是房间卫生...,neg
9,"非常糟糕的酒店,住了1天,停电两次,酒店装修的材料很劣质,甲醛的味道让人睁不开眼睛.地理位置...",neg


In [5]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]  
    return stopwords  
def classify_words(word_dict):  
    """词语分类,找出情感词、否定词、程度副词"""  
    # 读取情感字典文件  
    sen_word = dict()  
    not_word = dict()  
    degree_word = dict()  
    # 分类  
    for word in word_dict.keys(): 
        if word in sen_dict.keys() and word not in not_word_list and word not in degree_dic.keys():  
            # 找出分词结果中在情感字典中的词  
            sen_word[word_dict[word]] = sen_dict[word]  
        elif word+'\n' in not_word_list and word not in degree_dic.keys():  
            # 分词结果中在否定词列表中的词  
            not_word[word_dict[word]] = -1 
        elif word in degree_dic.keys():  
            # 分词结果中在程度副词中的词  
            degree_word[word_dict[word]] = degree_dic[word]   
    # 将分类结果返回  
    sen_num=len(sen_word)    
    degree_num=len(degree_word) 
    not_num=len(not_word)
    return sen_word,not_word,degree_word,sen_num,not_num,degree_num 
 
def list_to_dict(word_list):  
    """将分词后的列表转为字典，key为单词，value为单词在列表中的索引，索引相当于词语在文档中出现的位置"""  
    data = {}  
    for x in range(0, len(word_list)):  
        data[word_list[x]] = x  
    return data  
  
def get_init_weight(sen_word, not_word,degree_word):  
    # 权重初始化为1  
    W = 1  
    # 将情感字典的key转为list  
    sen_word_index_list = list(sen_word.keys())  
    if len(sen_word_index_list) == 0:  
        return W  
    # 获取第一个情感词的下标，遍历从0到此位置之间的所有词，找出程度词和否定词  
    for i in range(0, sen_word_index_list[0]):  
        if i in not_word.keys():  
            W *= -1  
        elif i in degree_word.keys():  
             # 更新权重，如果有程度副词，分值乘以程度副词的程度分值  
            W *= float(degree_word[i])  
    return W    
  
def socre_sentiment(sen_word, not_word,degree_word,seg_result): #degree_word  
    """计算得分"""  
    # 权重初始化为1  
    num=0;
    W = 1  
    score = 0  
    # 情感词下标初始化  
    sentiment_index = -1  
    # 情感词的位置下标集合  
    sentiment_index_list = list(sen_word.keys())  
    # 遍历分词结果(遍历分词结果是为了定位两个情感词之间的程度副词和否定词)  
    for i in range(0, len(seg_result)):  
        # 如果是情感词（根据下标是否在情感词分类结果中判断）  
        if i in sen_word.keys():  
            num=num+1;
            # 权重*情感词得分  
            score=score+ W * float(sen_word[i])  
            # 情感词下标加1，获取下一个情感词的位置  
            sentiment_index += 1  
            if sentiment_index < len(sentiment_index_list) - 1:  
                # 判断当前的情感词与下一个情感词之间是否有程度副词或否定词  
                for j in range(sentiment_index_list[sentiment_index], sentiment_index_list[sentiment_index + 1]):  
                    # 更新权重，如果有否定词，取反  
                    if j in not_word.keys():  
                        W = W*-1  
                    elif j in degree_word.keys():  
                        # 更新权重，如果有程度副词，分值变成程度副词的程度分值  
                        W *= float(degree_word[j])
                        W *= float(degree_word[j])
        # 定位到下一个情感词  
        if sentiment_index < len(sentiment_index_list) - 1:  
            i = sentiment_index_list[sentiment_index + 1]
        if num!=0:
            #考虑到了评论长度因素
            score=score/num  
    return score

In [6]:
from collections import defaultdict 
sen_file = open(r'BosonNLP_sentiment_score.txt', 'r', encoding='utf-8')  
# 获取字典文件内容  
sen_list = sen_file.readlines()  
# 创建情感字典  
sen_dict = defaultdict()  
# 读取字典文件每一行内容，将其转换为字典对象，key为情感词，value为对应的分值
for s in sen_list:  
    # 每一行内容根据空格分割，索引0是情感词，索引01是情感分值
    if (len(s.split(' '))==2):
        sen_dict[s.split(' ')[0]] = s.split(' ')[1]  

# 读取否定词文件  
not_word_file = open(r'notDic.txt', 'r+', encoding='utf-8-sig')  
# 由于否定词只有词，没有分值，使用list即可  
not_word_dict = defaultdict()  
not_word_list = not_word_file.readlines() 

#读取程度副词文件  
degree_file = open(r'degree.txt','r+')  
degree_list = degree_file.readlines()  
degree_dic = defaultdict()  
#程度副词与情感词处理方式一样，转为程度副词字典对象，key为程度副词，value为对应的程度值  
for d in degree_list:  
    degree_dic[d.split(',')[0]] = d.split(',')[1]  

sen_file.close()  
degree_file.close()  
not_word_file.close() 

In [7]:
stopwords = stopwordslist(r'stopword.txt')

#情感计算

In [8]:
data =xlrd.open_workbook(r'10000_hotel.xlsx')
file = xlwt.Workbook()
table_w = file.add_sheet('fenci',cell_overwrite_ok=True)
table = data.sheet_by_index(0) #第一个表格
nrows = table.nrows #表格的行数
ncols = table.ncols #表格的列数 这里只有一列

for i in tqdm_notebook(range(table.nrows)):
    num=0;
    #seg_list = jieba.cut(table.row(i)[0].value) #获取第1列的值 索引号不管行还是列还是表都是从0开始
    outstr='' 
    sen_num,not_num,degree_num,score = 0,0,0,0
     # 3.计算得分  
    if i > 0 :
        table_w.write(i,0,table.row(i)[0].value) 
        for n in re.split('，|。|！|？', table.row(i)[0].value):  #获取第1列的值 索引号不管行还是列还是表都是从0开始
            #print(n)
            seg_list =  jieba.lcut(n)
            seg_result = [] 
            for word in seg_list:  #在分词的里面
                if word not in stopwords:  
                    if word != '\t':  
                        outstr += word  
                        outstr += " "
                        seg_result.append(word)
            #print(seg_result)
            sen_word, not_word,degree_word,sen_num_,not_num_,degree_num_= classify_words(list_to_dict(seg_result))
            sen_num += sen_num_
            not_num += not_num_
            degree_num += degree_num_
            score+=socre_sentiment(sen_word, not_word, degree_word, seg_result)
        table_w.write(i,1,outstr) 
        table_w.write(i,2,score)
        table_w.write(i,3,not_num)
        table_w.write(i,4,degree_num)
        table_w.write(i,5,sen_num)
    else :
        table_w.write(0,0,"原词") #第一行第一列写入“原词”
        table_w.write(0,1,"分词结果") 
        table_w.write(0,2,"score") 
        table_w.write(0,3,"num1")
        table_w.write(0,4,"num2")
        table_w.write(0,5,"num3")
file.save(r'result.xls')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.887 seconds.
Prefix dict has been built succesfully.


In [9]:
y = pd.read_csv('10000_hotel.csv', encoding='ansi')['sentiment']
le = LabelEncoder() #0-1
y=le.fit_transform(y)
y = pd.DataFrame(y, columns=['label'])
data = pd.read_excel("result.xls")

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# 过滤部分特征 cbow
token_pattern = u'(?u)\\b[^\\d\\W]\\w+\\b'

vectorizer = CountVectorizer(min_df = 3,
               token_pattern=token_pattern,
               stop_words=frozenset(stopwords))

x_ = data['分词结果']

# 文本向量化
x = vectorizer.fit_transform(x_.values.astype('U'))

In [12]:
#降维
ch2 = SelectKBest(chi2,k=1000)
x_train_ = ch2.fit_transform(x,y)

x_train_ = pd.DataFrame(x_train_.toarray())
x_train_ = x_train_.set_index(data.index)
x = pd.concat([data.iloc[:,2:],x_train_],axis=1).astype(float)

#lgb

In [13]:
def Lgb_Classifier(train_x, train_y,test_x,test_y, params_lgb=None):
    import lightgbm as lgb
    lgb_trn = lgb.Dataset(
                        data=train_x,
                        label=train_y,
                        free_raw_data=True)
    lgb_val = lgb.Dataset(
                        data=test_x,
                        label=test_y,
                        free_raw_data=True)
    if params_lgb==None:
        params_lgb = {'boosting_type': 'gbdt','objective': 'binary','save_binary': True,
                  'metric': 'binary_logloss', 'cat_smooth': 30,'is_unbalance': True,'boost_from_average': False,
                  'num_leaves': 168, 'learning_rate': 0.01}
    fit_params_lgb = {'num_boost_round': 10000, 'verbose_eval':100,'early_stopping_rounds':100}
    
    lgb_reg = lgb.train(params=params_lgb, 
                        train_set=lgb_trn, 
                        **fit_params_lgb,
                        valid_sets=[lgb_trn, lgb_val])
    return lgb_reg

def classify(pred):
    l = []
    for n in pred:
        if n>0.5:
            l.append(1)
        else:
            l.append(0)
    return l

In [111]:
import lightgbm as lgb
n_splits=10
y_pred = []
y_true = []
sfold = ShuffleSplit(n_splits=n_splits, test_size=0.3)
for train_idx, val_idx in sfold.split(x):
    train_x = x.iloc[train_idx,:]
    train_y = y['label'].iloc[train_idx]
    #print(train_x,train_y)
    test_x = x.iloc[val_idx,:]
    test_y = y['label'].iloc[val_idx]
    model = Lgb_Classifier(train_x, train_y, test_x, test_y)
    y_pred.extend(classify(model.predict(test_x)))
    y_true.extend(test_y)
    
print(classification_report(y_true, y_pred))

Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.404741	valid_1's binary_logloss: 0.458524
[200]	training's binary_logloss: 0.281896	valid_1's binary_logloss: 0.376494
[300]	training's binary_logloss: 0.199975	valid_1's binary_logloss: 0.325447
[400]	training's binary_logloss: 0.142892	valid_1's binary_logloss: 0.29265
[500]	training's binary_logloss: 0.103929	valid_1's binary_logloss: 0.275359
[600]	training's binary_logloss: 0.0776465	valid_1's binary_logloss: 0.268248
[700]	training's binary_logloss: 0.0590542	valid_1's binary_logloss: 0.266634
Early stopping, best iteration is:
[696]	training's binary_logloss: 0.0596778	valid_1's binary_logloss: 0.266544
Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.409641	valid_1's binary_logloss: 0.457073
[200]	training's binary_logloss: 0.286135	valid_1's binary_logloss: 0.373302
[300]	training's binary_logloss: 0.201867	valid_1's binary_logloss: 0

In [18]:
# larger model
from keras import backend as K
import tensorflow as tf

def create_larger(len_col):
    # create model
    model = Sequential()
    model.add(Dense(len_col, input_dim=len_col, init='normal', activation='relu'))
    from keras.layers import Dropout
    model.add(Dropout(0.25))
    model.add(Dense(300, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(len_col, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

#神经网络

In [20]:
n_splits=10
predictions = x.columns
sfold = ShuffleSplit(n_splits=n_splits, test_size=0.3)
y_pred = []
y_true = []
for train_idx, val_idx in sfold.split(x):
    train_x = x[predictions].iloc[train_idx]
    train_y = y['label'].iloc[train_idx]
    #print(train_x,train_y)
    test_x = x[predictions].iloc[val_idx]
    test_y = y['label'].iloc[val_idx]
    len_col = train_x.shape[1]
    nn_model = create_larger(len_col)
    nn_model.fit(train_x, train_y, nb_epoch=20, batch_size=512,  verbose=2)

    y_pred.extend(classify(nn_model.predict(test_x).T[0].tolist()))
    y_true.extend(test_y)
print(classification_report(y_true,y_pred))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1004, input_dim=1004, activation="relu", kernel_initializer="normal")`
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
 - 2s - loss: 0.8270 - acc: 0.6591
Epoch 2/20
 - 1s - loss: 0.5308 - acc: 0.7924
Epoch 3/20
 - 1s - loss: 0.4646 - acc: 0.8246
Epoch 4/20
 - 1s - loss: 0.4097 - acc: 0.8573
Epoch 5/20
 - 1s - loss: 0.3717 - acc: 0.8733
Epoch 6/20
 - 1s - loss: 0.3661 - acc: 0.8881
Epoch 7/20
 - 1s - loss: 0.3548 - acc: 0.8931
Epoch 8/20
 - 1s - loss: 0.3172 - acc: 0.8989
Epoch 9/20
 - 1s - loss: 0.3003 - acc: 0.9047
Epoch 10/20
 - 1s - loss: 0.2758 - acc: 0.9147
Epoch 11/20
 - 1s - loss: 0.2630 - acc: 0.9157
Epoch 12/20
 - 1s - loss: 0.2487 - acc: 0.9234
Epoch 13/20
 - 1s - loss: 0.2296 - acc: 0.9317
Epoch 14/20
 - 1s - loss: 0.2152 - acc: 0.9386
Epoch 15/20
 - 1s - loss: 0.2217 - acc: 0.9389
Epoch 16/20
 - 1s - loss: 0.2134 - acc: 0.9409
Epoch 17/20
 - 1s - loss: 0.1926 - acc: 0.9469
Epoch 18/20
 - 1s - loss: 0.1659 - acc: 0.9594
Epoch 19/20
 - 1s - loss: 0.1563 - acc: 0.9667
Epoch 20/20
 - 1s - loss: 0.1431 - acc: 0.9727
Epoch 1/20
 - 2s - loss: 0.8657 - acc: 0.6497
Epoch 2/20
 - 1s - loss

#多模

In [118]:
n_splits=10
predictions = x_train.columns
sfold = ShuffleSplit(n_splits=n_splits, test_size=0.3)
y_pred = []
y_true = []
for train_idx, val_idx in sfold.split(x_train):
    train_x = x[predictions].iloc[train_idx]
    train_y = y['label'].iloc[train_idx]
    #print(train_x,train_y)
    test_x = x[predictions].iloc[val_idx]
    test_y = y['label'].iloc[val_idx]
    lgb_model = Lgb_Classifier(train_x, train_y, test_x, test_y)
    xgb_model = xgb.XGBClassifier()
    xgb_model.fit(train_x, train_y,eval_set=[(test_x, test_y)],verbose=100,early_stopping_rounds=50)
    cbt_model = cbt.CatBoostClassifier()
    cbt_model.fit(train_x, train_y,eval_set=[(test_x, test_y)],verbose=100,early_stopping_rounds=50)
    len_col = train_x.shape[1]
    nn_model = create_larger(len_col)
    nn_model.fit(train_x, train_y, nb_epoch=20, batch_size=512,  verbose=2)

    y_pred.extend(classify(np.array([lgb_model.predict(test_x), [n[1] for n in xgb_model.predict_proba(test_x)], 
                                    [n[1] for n in cbt_model.predict_proba(test_x)], nn_model.predict(test_x).T[0].tolist()]).sum(axis=0)/4))
    y_true.extend(test_y)
print(classification_report(y_true,y_pred))

Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.403839	valid_1's binary_logloss: 0.463303
[200]	training's binary_logloss: 0.278333	valid_1's binary_logloss: 0.381955
[300]	training's binary_logloss: 0.197095	valid_1's binary_logloss: 0.332232
[400]	training's binary_logloss: 0.140718	valid_1's binary_logloss: 0.300341
[500]	training's binary_logloss: 0.102576	valid_1's binary_logloss: 0.281665
[600]	training's binary_logloss: 0.0765909	valid_1's binary_logloss: 0.272084
[700]	training's binary_logloss: 0.0584244	valid_1's binary_logloss: 0.269387
[800]	training's binary_logloss: 0.0448643	valid_1's binary_logloss: 0.269867
Early stopping, best iteration is:
[735]	training's binary_logloss: 0.0532381	valid_1's binary_logloss: 0.269102
[0]	validation_0-error:0.211333
Will train until validation_0-error hasn't improved in 50 rounds.
[99]	validation_0-error:0.148
Learning rate set to 0.095716
0:	learn: 0.6356285	test: 0.6350615	best: 0.6350

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1004, input_dim=1004, activation="relu", kernel_initializer="normal")`
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
 - 2s - loss: 0.7781 - acc: 0.6824
Epoch 2/20
 - 1s - loss: 0.5084 - acc: 0.7991
Epoch 3/20
 - 1s - loss: 0.4430 - acc: 0.8401
Epoch 4/20
 - 1s - loss: 0.4038 - acc: 0.8613
Epoch 5/20
 - 1s - loss: 0.3623 - acc: 0.8801
Epoch 6/20
 - 1s - loss: 0.3332 - acc: 0.8877
Epoch 7/20
 - 1s - loss: 0.3047 - acc: 0.9021
Epoch 8/20
 - 1s - loss: 0.2988 - acc: 0.9073
Epoch 9/20
 - 1s - loss: 0.2664 - acc: 0.9177
Epoch 10/20
 - 1s - loss: 0.2460 - acc: 0.9269
Epoch 11/20
 - 1s - loss: 0.2500 - acc: 0.9301
Epoch 12/20
 - 1s - loss: 0.2504 - acc: 0.9360
Epoch 13/20
 - 1s - loss: 0.2185 - acc: 0.9410
Epoch 14/20
 - 1s - loss: 0.2035 - acc: 0.9463
Epoch 15/20
 - 1s - loss: 0.1787 - acc: 0.9560
Epoch 16/20
 - 1s - loss: 0.1618 - acc: 0.9630
Epoch 17/20
 - 1s - loss: 0.1453 - acc: 0.9690
Epoch 18/20
 - 1s - loss: 0.1367 - acc: 0.9741
Epoch 19/20
 - 1s - loss: 0.1314 - acc: 0.9753
Epoch 20/20
 - 1s - loss: 0.1458 - acc: 0.9744
Training until validation scores don't improve for 100 rounds
[100]	t

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1004, input_dim=1004, activation="relu", kernel_initializer="normal")`
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
 - 2s - loss: 0.7022 - acc: 0.7153
Epoch 2/20
 - 1s - loss: 0.4779 - acc: 0.8030
Epoch 3/20
 - 1s - loss: 0.4184 - acc: 0.8403
Epoch 4/20
 - 1s - loss: 0.3579 - acc: 0.8704
Epoch 5/20
 - 1s - loss: 0.3177 - acc: 0.8860
Epoch 6/20
 - 1s - loss: 0.2904 - acc: 0.8966
Epoch 7/20
 - 1s - loss: 0.3350 - acc: 0.8983
Epoch 8/20
 - 1s - loss: 0.2630 - acc: 0.9121
Epoch 9/20
 - 1s - loss: 0.2395 - acc: 0.9211
Epoch 10/20
 - 1s - loss: 0.2253 - acc: 0.9254
Epoch 11/20
 - 1s - loss: 0.1981 - acc: 0.9387
Epoch 12/20
 - 1s - loss: 0.1910 - acc: 0.9427
Epoch 13/20
 - 1s - loss: 0.1584 - acc: 0.9543
Epoch 14/20
 - 1s - loss: 0.1497 - acc: 0.9600
Epoch 15/20
 - 1s - loss: 0.1290 - acc: 0.9673
Epoch 16/20
 - 1s - loss: 0.1147 - acc: 0.9736
Epoch 17/20
 - 1s - loss: 0.1050 - acc: 0.9741
Epoch 18/20
 - 1s - loss: 0.1428 - acc: 0.9727
Epoch 19/20
 - 1s - loss: 0.1179 - acc: 0.9770
Epoch 20/20
 - 1s - loss: 0.1120 - acc: 0.9760
Training until validation scores don't improve for 100 rounds
[100]	t

...

Epoch 1/20
 - 2s - loss: 0.7292 - acc: 0.7014
Epoch 2/20
 - 1s - loss: 0.5314 - acc: 0.8034
Epoch 3/20
 - 1s - loss: 0.4811 - acc: 0.8367
Epoch 4/20
 - 1s - loss: 0.4087 - acc: 0.8464
Epoch 5/20
 - 1s - loss: 0.3692 - acc: 0.8727
Epoch 6/20
 - 1s - loss: 0.3274 - acc: 0.8887
Epoch 7/20
 - 1s - loss: 0.3313 - acc: 0.8956
Epoch 8/20
 - 1s - loss: 0.2891 - acc: 0.9053
Epoch 9/20
 - 1s - loss: 0.2875 - acc: 0.9064
Epoch 10/20
 - 1s - loss: 0.2649 - acc: 0.9161
Epoch 11/20
 - 1s - loss: 0.2395 - acc: 0.9254
Epoch 12/20
 - 1s - loss: 0.2424 - acc: 0.9336
Epoch 13/20
 - 1s - loss: 0.2357 - acc: 0.9374
Epoch 14/20
 - 1s - loss: 0.2662 - acc: 0.9436
Epoch 15/20
 - 1s - loss: 0.2286 - acc: 0.9444
Epoch 16/20
 - 1s - loss: 0.1856 - acc: 0.9511
Epoch 17/20
 - 1s - loss: 0.1604 - acc: 0.9614
Epoch 18/20
 - 1s - loss: 0.1492 - acc: 0.9680
Epoch 19/20
 - 1s - loss: 0.1330 - acc: 0.9736
Epoch 20/20
 - 1s - loss: 0.1233 - acc: 0.9759
Training until validation scores don't improve for 100 rounds
[100]	t

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1004, input_dim=1004, activation="relu", kernel_initializer="normal")`
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
 - 2s - loss: 0.8659 - acc: 0.6744
Epoch 2/20
 - 1s - loss: 0.4969 - acc: 0.8030
Epoch 3/20
 - 1s - loss: 0.4341 - acc: 0.8406
Epoch 4/20
 - 1s - loss: 0.3834 - acc: 0.8610
Epoch 5/20
 - 1s - loss: 0.3472 - acc: 0.8767
Epoch 6/20
 - 1s - loss: 0.3317 - acc: 0.8897
Epoch 7/20
 - 1s - loss: 0.2959 - acc: 0.9011
Epoch 8/20
 - 1s - loss: 0.2728 - acc: 0.9053
Epoch 9/20
 - 1s - loss: 0.2654 - acc: 0.9136
Epoch 10/20
 - 1s - loss: 0.2455 - acc: 0.9173
Epoch 11/20
 - 1s - loss: 0.2845 - acc: 0.9210
Epoch 12/20
 - 1s - loss: 0.2157 - acc: 0.9337
Epoch 13/20
 - 1s - loss: 0.1987 - acc: 0.9387
Epoch 14/20
 - 1s - loss: 0.1763 - acc: 0.9533
Epoch 15/20
 - 1s - loss: 0.1615 - acc: 0.9567
Epoch 16/20
 - 1s - loss: 0.1498 - acc: 0.9644
Epoch 17/20
 - 1s - loss: 0.1410 - acc: 0.9687
Epoch 18/20
 - 1s - loss: 0.1254 - acc: 0.9724
Epoch 19/20
 - 1s - loss: 0.1103 - acc: 0.9770
Epoch 20/20
 - 1s - loss: 0.1141 - acc: 0.9791
Training until validation scores don't improve for 100 rounds
[100]	t

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1004, input_dim=1004, activation="relu", kernel_initializer="normal")`
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
 - 3s - loss: 0.7329 - acc: 0.6973
Epoch 2/20
 - 1s - loss: 0.4867 - acc: 0.8010
Epoch 3/20
 - 1s - loss: 0.4284 - acc: 0.8400
Epoch 4/20
 - 1s - loss: 0.3816 - acc: 0.8569
Epoch 5/20
 - 1s - loss: 0.3368 - acc: 0.8787
Epoch 6/20
 - 1s - loss: 0.3200 - acc: 0.8844
Epoch 7/20
 - 1s - loss: 0.2910 - acc: 0.8963
Epoch 8/20
 - 1s - loss: 0.2773 - acc: 0.9033
Epoch 9/20
 - 1s - loss: 0.2555 - acc: 0.9139
Epoch 10/20
 - 1s - loss: 0.2766 - acc: 0.9206
Epoch 11/20
 - 1s - loss: 0.2689 - acc: 0.9277
Epoch 12/20
 - 1s - loss: 0.2272 - acc: 0.9327
Epoch 13/20
 - 1s - loss: 0.1932 - acc: 0.9399
Epoch 14/20
 - 1s - loss: 0.2014 - acc: 0.9504
Epoch 15/20
 - 1s - loss: 0.1699 - acc: 0.9529
Epoch 16/20
 - 1s - loss: 0.1523 - acc: 0.9596
Epoch 17/20
 - 1s - loss: 0.1295 - acc: 0.9677
Epoch 18/20
 - 1s - loss: 0.1225 - acc: 0.9756
Epoch 19/20
 - 1s - loss: 0.1503 - acc: 0.9716
Epoch 20/20
 - 1s - loss: 0.1180 - acc: 0.9734
             precision    recall  f1-score   support

          0    